In [18]:
import requests
import json
from pathlib import Path

In [3]:
response = requests.get('https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=050000&semester_id=20251')
response.text

'[{"nama":"Kab. Bojonegoro","kode_wilayah":"050500  ","id_level_wilayah":2,"mst_kode_wilayah":"050000  ","induk_provinsi":"Prov. Jawa Timur","kode_wilayah_induk_provinsi":"050000  ","induk_kabupaten":null,"kode_wilayah_induk_kabupaten":null,"kirim_total":13,"sekolah_total":13,"persen":100},{"nama":"Kab. Bondowoso","kode_wilayah":"052200  ","id_level_wilayah":2,"mst_kode_wilayah":"050000  ","induk_provinsi":"Prov. Jawa Timur","kode_wilayah_induk_provinsi":"050000  ","induk_kabupaten":null,"kode_wilayah_induk_kabupaten":null,"kirim_total":5,"sekolah_total":5,"persen":100},{"nama":"Kab. Gresik","kode_wilayah":"050100  ","id_level_wilayah":2,"mst_kode_wilayah":"050000  ","induk_provinsi":"Prov. Jawa Timur","kode_wilayah_induk_provinsi":"050000  ","induk_kabupaten":null,"kode_wilayah_induk_kabupaten":null,"kirim_total":8,"sekolah_total":8,"persen":100},{"nama":"Kab. Jember","kode_wilayah":"052400  ","id_level_wilayah":2,"mst_kode_wilayah":"050000  ","induk_provinsi":"Prov. Jawa Timur","kode

In [5]:
data =json.loads(response.text)
data

[{'nama': 'Kab. Bojonegoro',
  'kode_wilayah': '050500  ',
  'id_level_wilayah': 2,
  'mst_kode_wilayah': '050000  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': None,
  'kode_wilayah_induk_kabupaten': None,
  'kirim_total': 13,
  'sekolah_total': 13,
  'persen': 100},
 {'nama': 'Kab. Bondowoso',
  'kode_wilayah': '052200  ',
  'id_level_wilayah': 2,
  'mst_kode_wilayah': '050000  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': None,
  'kode_wilayah_induk_kabupaten': None,
  'kirim_total': 5,
  'sekolah_total': 5,
  'persen': 100},
 {'nama': 'Kab. Gresik',
  'kode_wilayah': '050100  ',
  'id_level_wilayah': 2,
  'mst_kode_wilayah': '050000  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': None,
  'kode_wilayah_induk_kabupaten': None,
  'kirim_total': 8,
  'sekolah_total': 8,
  'persen': 100},
 {'nama': 'Ka

In [10]:
with open('../data/test.json', 'w') as f:
    json.dump(data, f, indent=4)

In [11]:
import pandas as pd

In [12]:
df_all_province = pd.DataFrame(data)
df_all_province

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,kirim_total,sekolah_total,persen
0,Kab. Bojonegoro,050500,2,050000,Prov. Jawa Timur,050000,None,None,13,13,100
1,Kab. Bondowoso,052200,2,050000,Prov. Jawa Timur,050000,None,None,5,5,100
2,Kab. Gresik,050100,2,050000,Prov. Jawa Timur,050000,None,None,8,8,100
3,Kab. Jember,052400,2,050000,Prov. Jawa Timur,050000,None,None,10,10,100
4,Kab. Jombang,050400,2,050000,Prov. Jawa Timur,050000,None,None,16,16,100
5,Kab. Kediri,051300,2,050000,Prov. Jawa Timur,050000,None,None,26,26,100
6,Kab. Lamongan,050700,2,050000,Prov. Jawa Timur,050000,None,None,9,9,100
7,Kab. Lumajang,052100,2,050000,Prov. Jawa Timur,050000,None,None,8,8,100
8,Kab. Madiun,050800,2,050000,Prov. Jawa Timur,050000,None,None,11,11,100
9,Kab. Magetan,051000,2,050000,Prov. Jawa Timur,050000,None,None,8,8,100


In [ ]:
for idx, row in df_all_province.iterrows():
    kode_kab = row['kode_wilayah']
    nama_kab = row['nama']
    url_kab = f'https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=2&kode_wilayah={kode_kab.strip()}&semester_id=20251'
    response_kab = requests.get(url_kab)
    data_kab = json.loads(response_kab.text)
    df_kab = pd.DataFrame(data_kab)

    Path(f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}').mkdir(parents=True, exist_ok=True)

    for idx2, row2 in df_kab.iterrows():
        kode_kec = row2['kode_wilayah']
        nama_kec = row2['nama']

        url_kec = f'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb?id_level_wilayah=3&kode_wilayah={kode_kec.strip()}&semester_id=20251'
        response_kec = requests.get(url_kec)
        data_kec = json.loads(response_kec.text)
        df_kec = pd.DataFrame(data_kec)

        Path(f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}/{nama_kec}').mkdir(parents=True, exist_ok=True)

        for idx3, row3, in df_kec.iterrows():
            nama_sekolah = row3['nama']
            kode_sekolah_enkrip = row3['sekolah_id_enkrip']
            

            url_sekolah = f'https://dapo.kemendikdasmen.go.id/rekap/sekolahDetail?semester_id=20251&sekolah_id={kode_sekolah_enkrip.strip()}'

            response_sekolah = requests.get(url_sekolah)
            data_sekolah = json.loads(response_sekolah.text)

            with open(f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}/{nama_kec}/{nama_sekolah}.json', 'w') as f:
                json.dump(data_sekolah, f, indent=4)

[{'rombel': 14, 'guru_kelas': 11, 'guru_matematika': 0, 'guru_bahasa_indonesia': 0, 'guru_bahasa_inggris': 0, 'guru_sejarah_indonesia': 0, 'guru_pkn': 0, 'guru_penjaskes': 1, 'guru_agama_budi_pekerti': 0, 'guru_seni_budaya': 0, 'ptk_laki': 2, 'ptk_perempuan': 11, 'pegawai_laki': 1, 'pegawai_perempuan': 2, 'pd_kelas_1_laki': 9, 'pd_kelas_1_perempuan': 3, 'pd_kelas_2_laki': 5, 'pd_kelas_2_perempuan': 2, 'pd_kelas_3_laki': 5, 'pd_kelas_3_perempuan': 2, 'pd_kelas_4_laki': 5, 'pd_kelas_4_perempuan': 2, 'pd_kelas_5_laki': 5, 'pd_kelas_5_perempuan': 1, 'pd_kelas_6_laki': 2, 'pd_kelas_6_perempuan': 6, 'pd_kelas_7_laki': 5, 'pd_kelas_7_perempuan': 4, 'pd_kelas_8_laki': 4, 'pd_kelas_8_perempuan': 4, 'pd_kelas_9_laki': 3, 'pd_kelas_9_perempuan': 1, 'pd_kelas_10_laki': 4, 'pd_kelas_10_perempuan': 3, 'pd_kelas_11_laki': 5, 'pd_kelas_11_perempuan': 0, 'pd_kelas_12_laki': 1, 'pd_kelas_12_perempuan': 2, 'pd_kelas_13_laki': 0, 'pd_kelas_13_perempuan': 0, 'jumlah_kirim': 11, 'ptk': 13, 'pegawai': 3, 'pd

In [ ]:
# ambil data tiap kota/kabupaten
for kode_kab in df_all_province['kode_wilayah']:
    response_kota = requests.get(f'https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=2&kode_wilayah={kode_kab}&semester_id=20251')
    data = json.loads(response.text)
    df_kota = pd.DataFrame(data)

    
    Path(f'../data/target/{}')

    # ambil data tiap kecamatan
    for kode_kec in df_kota['kode_wilayah']:
        response_kec = requests.get(f'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb?id_level_wilayah=3&kode_wilayah={kode_kec}&semester_id=20251')
        data_kecamatan = json.loads(response_kec.text)
        df_kec = pd.DataFrame(data_kecamatan)

        df_kec.to_json(f'../data/test-yes/{kode_kec}.json')


In [16]:
data

[{'nama': 'Kec. Cerme',
  'kode_wilayah': '050105  ',
  'id_level_wilayah': 3,
  'mst_kode_wilayah': '050100  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': 'Kab. Gresik',
  'kode_wilayah_induk_kabupaten': '050100  ',
  'kirim_total': 1,
  'sekolah_total': 1,
  'persen': 100},
 {'nama': 'Kec. Driyorejo',
  'kode_wilayah': '050102  ',
  'id_level_wilayah': 3,
  'mst_kode_wilayah': '050100  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': 'Kab. Gresik',
  'kode_wilayah_induk_kabupaten': '050100  ',
  'kirim_total': 2,
  'sekolah_total': 2,
  'persen': 100},
 {'nama': 'Kec. Dukun',
  'kode_wilayah': '050114  ',
  'id_level_wilayah': 3,
  'mst_kode_wilayah': '050100  ',
  'induk_provinsi': 'Prov. Jawa Timur',
  'kode_wilayah_induk_provinsi': '050000  ',
  'induk_kabupaten': 'Kab. Gresik',
  'kode_wilayah_induk_kabupaten': '050100  ',
  'kirim_total': 1,
  'sekolah_total